In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.26635313034057617
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 10.746297891936528
200000 18.004388937198694
300000 18.356007408441464
400000 24.417346132383457
500000 26.41541993756644
600000 28.233535296334136
700000 27.767365459178404
800000 29.39254370662949
900000 35.23699065029157
1000000 40.1444799170752
1100000 45.75379042185449
1200000 40.56671876631197
1300000 32.08869480915565
1400000 58.10357602105249
1500000 56.19838350407482
1600000 58.02562451868031
1700000 58.72961069591357
1800000 60.57378265019104
1900000 61.80726597049792
2000000 52.729000364234906
2100000 57.10929271174341
2200000 58.13674287618221
2300000 57.943854257521004
2400000 42.75771723301629
2500000 23.883816787998093
2600000 54.946325318604174
2700000 56.861503374995685
2800000 50.33335942687249
2900000 48.180205132374915
3000000 45.099092

29800000 52.86080810109526
29900000 49.85432200093109
30000000 50.86943636858105
30100000 24.552583396194297
30200000 37.59112460014477
30300000 49.61012064038655
30400000 48.76054846274429
30500000 53.00495511252594
30600000 48.98070109414661
30700000 52.4019371157187
30800000 51.26546678411821
30900000 54.460553521236704
31000000 52.97301810398864
31100000 26.402132649854096
31200000 46.865360813628165
31300000 55.919456308303324
31400000 50.21372870019446
31500000 46.142536244756485
31600000 25.059041504839794
31700000 50.38510243869442
31800000 50.873801894326135
31900000 50.798828319003114
32000000 50.241093353766466
32100000 50.67610159432321
32200000 48.940404867073
32300000 47.19046846394149
32400000 46.70167996602691
32500000 26.37582571719426
32600000 37.37379474095
32700000 44.0407658862997
32800000 47.167641104351794
32900000 48.94509534114722
33000000 49.79507490441883
33100000 48.10814615965019
33200000 49.06956491992296
33300000 51.87314310515891
33400000 49.007141676310

59900000 47.37629009872575
60000000 49.542348763687286
60100000 46.41626125058398
60200000 50.28179688070164
60300000 47.40128951873302
60400000 49.115789360477116
60500000 48.6333821741446
60600000 39.10326932036799
60700000 27.90728200585822
60800000 47.87430177487721
60900000 44.110947378027625
61000000 47.74877612500776
61100000 42.43515243372499
61200000 45.714738706480766
61300000 40.86024764092535
61400000 14.357982130932019
61500000 44.69583898943473
61600000 41.200288428017444
61700000 43.99936507478855
61800000 42.277144282076264
61900000 43.745671221362805
62000000 45.01824158550265
62100000 38.14066740008347
62200000 30.18554551594953
62300000 49.173290589427076
62400000 47.608475471473156
62500000 46.253180718483456
62600000 47.54579883979853
62700000 49.265079108554666
62800000 49.641879971957
62900000 49.31897052606493
63000000 46.861698172112455
63100000 19.842655786457385
63200000 46.539694728998406
63300000 46.76721018363148
63400000 44.57331904914069
63500000 43.5432

90000000 40.92677270982683
90100000 38.117900325445554
90200000 38.936355897229284
90300000 12.584041696885317
90400000 39.34130457624387
90500000 39.40271231922487
90600000 37.616671404580295
90700000 39.22570792323732
90800000 20.96091881064274
90900000 36.07468092596801
91000000 41.93290792977978
91100000 40.98966655204204
91200000 44.399392750460365
91300000 42.96668082182256
91400000 42.136962442072324
91500000 43.80405867056258
91600000 45.25803678360229
91700000 42.462926058479304
91800000 43.810366801740976
91900000 41.891570520372845
92000000 42.154702716913334
92100000 34.536267031780234
92200000 19.2255226678616
92300000 39.275331716683766
92400000 38.25799671755873
92500000 39.49836310298722
92600000 41.01536294888216
92700000 9.621484436231997
92800000 41.76779548869278
92900000 38.77935696461014
93000000 39.4509821689437
93100000 38.03780936186843
93200000 18.30560071128232
93300000 38.513506899233896
93400000 42.212182678421634
93500000 41.558849915133194
93600000 42.617

119400000 34.046448356433764
119500000 19.064024574367487
119600000 35.530282543263844
119700000 36.31280029978648
119800000 34.85637436398808
119900000 38.12606550131506
120000000 38.93243611106098
120100000 39.190345413369045
120200000 40.64994146121082
120300000 40.03541977630153
120400000 40.337616383398085
120500000 42.204221929130384
120600000 40.96562433819898
120700000 39.60658310800596
120800000 38.204927312121114
120900000 38.8431342872019
121000000 36.83347495685233
121100000 37.47259715431711
121200000 28.401712618046073
121300000 22.368920039487403
121400000 35.77749026253378
121500000 28.797103617301993
121600000 19.395689878205758
121700000 35.36695106791549
121800000 31.883002009488287
121900000 21.087580828717112
122000000 37.55693581762492
122100000 37.57803725176544
122200000 36.035641960702655
122300000 37.6373004410264
122400000 38.43656934650288
122500000 40.52653053150281
122600000 40.32463075777651
122700000 40.573255631635135
122800000 39.72536576951726
1229000

...

493900000 3.3339739305109806
494000000 7.15488881378719
494100000 6.487638619475793
494200000 7.124730978374318
494300000 6.336590063203424
494400000 5.849459616811949
494500000 4.706887548281164
494600000 6.193366231592185
494700000 6.60875559930707
494800000 6.6095427987770226
494900000 7.289950444528249
495000000 7.035989907588505
495100000 3.21794753376394
495200000 6.863104503190508
495300000 7.222653406458449
495400000 6.572877177266754
495500000 6.170758163596254
495600000 5.440977756504221
495700000 6.492959596059807
495800000 5.037179570466643
495900000 6.6008906776575484
496000000 6.62499424407017
496100000 7.252917551624002
496200000 6.5757469198730485
496300000 3.269414894450152
496400000 6.725008028762287
496500000 6.769176939031636
496600000 6.372519067034239
496700000 6.140273920362845
496800000 5.561873724082173
496900000 6.0156909879950495
497000000 5.021411415547804
497100000 6.913864344153488
497200000 7.180663779641898
497300000 6.591271344852877
497400000 6.3124588

523200000 4.760912584812354
523300000 5.574785197954516
523400000 5.327176891713064
523500000 4.930015562060988
523600000 5.464456706148608
523700000 4.722921191587331
523800000 4.988870171606043
523900000 3.847265607699932
524000000 2.6953771999406912
524100000 3.649704472807069
524200000 4.805149886393465
524300000 4.828449226647836
524400000 4.812090024152116
524500000 5.387495869979718
524600000 4.9017235089492415
524700000 4.767321227949903
524800000 5.135721326550411
524900000 4.522693285967269
525000000 4.790827227027876
525100000 3.7830057357423783
525200000 2.6803420602637944
525300000 3.702267577491268
525400000 4.518072099317696
525500000 4.475793106296844
525600000 4.555356730379217
525700000 5.307711154896326
525800000 4.967322736930853
525900000 4.852878318025601
526000000 5.046284715803454
526100000 5.006313250183845
526200000 4.569503296754302
526300000 3.853116722344766
526400000 1.8386123954841396
526500000 4.288761631485442
526600000 4.724845664186489
526700000 4.579

552100000 4.376180748489106
552200000 4.135793590379316
552300000 4.164099477410582
552400000 4.152198431607566
552500000 2.9076599326728845
552600000 3.2625547039549425
552700000 3.6028727683133766
552800000 3.524779622078361
552900000 2.574819037250892
553000000 3.5395817990218204
553100000 3.6485929136983213
553200000 2.9829257967594396
553300000 4.1174706503950045
553400000 4.013490949858958
553500000 4.00896296832969
553600000 4.115088144639181
553700000 2.5227595089310846
553800000 3.564211515420728
553900000 3.5255696399331833
554000000 3.573736321911484
554100000 2.260911828512492
554200000 3.4564658662585863
554300000 3.519600608617493
554400000 3.323988276785704
554500000 3.8973216439186036
554600000 3.976397792340698
554700000 4.212144772448003
554800000 3.882870688177214
554900000 2.3005507555871487
555000000 3.721159145761422
555100000 3.9791611943846807
555200000 3.119989517069612
555300000 2.116622355666766
555400000 3.378568563767621
555500000 3.867021601536358
55560000

579600000 2.138126222090975
579700000 1.7911102140261081
579800000 2.2312484896683613
579900000 2.006476679348564
580000000 2.286779270805288
580100000 2.3818308002538027
580200000 2.250353212917909
580300000 1.7920753832769254
580400000 1.901595113866246
580500000 1.903428676526971
580600000 2.721349540248579
580700000 2.5132077191454907
580800000 1.816501376217143
580900000 1.851777841657171
581000000 2.1177520047848764
581100000 1.8297611073034508
581200000 2.5040714371025183
581300000 2.518931722168946
581400000 1.8913099628148495
581500000 1.9428026956195898
581600000 1.831822246141509
581700000 1.7432595534390123
581800000 2.556310183099731
581900000 2.716397091981161
582000000 1.6765831358965972
582100000 1.8648420402026873
582200000 1.8202475802236437
582300000 1.986459824154459
582400000 2.444222537890036
582500000 2.5133769521170914
582600000 1.678073528016556
582700000 2.163084060150372
582800000 1.782135313315642
582900000 1.6993701748802947
583000000 2.4394213461013305
583

608200000 0.9881345879522788
608300000 1.1926299120302002
608400000 1.010034242271838
608500000 1.1727182737490138
608600000 0.7240709563582937
608700000 1.3577985072452936
608800000 1.1746894831471293
608900000 1.0292888598698793
609000000 0.8951037807284402
609100000 1.220173385899453
609200000 0.782265313301299
609300000 1.1279166251458532
609400000 1.1109119202475284
609500000 0.9697402920575162
609600000 0.9543081491561896
609700000 1.126754928009534
609800000 0.6823772909112334
609900000 1.120471917762718
610000000 1.260080223869125
610100000 0.7761308817280809
610200000 0.8759560006730145
610300000 1.0979247146392306
610400000 0.7944451529387931
610500000 1.0781244101779415
610600000 1.099399677507068
610700000 0.7313238771428584
610800000 1.036694756358434
610900000 1.0731650389176561
611000000 0.672535608295234
611100000 1.0779440202844333
611200000 1.1942685680014908
611300000 0.6462870259493113
611400000 0.9414590413851222
611500000 1.045921558928968
611600000 0.794854057808